In [1]:
import torch
from torch import nn
import numpy as np
from defconv.models import ConvNet, DeformConvNet
from argus import Model
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [2]:
transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ])

In [3]:
mnist_trainset = DataLoader(datasets.MNIST(root='./data', train=True, download=True, transform=transform),
                           batch_size=64,shuffle=True)
mnist_valset = DataLoader(datasets.MNIST(root='./data', train=False, download=True, transform=transform),
                         batch_size=64, shuffle=True)

In [4]:
# in_channels = 1
# channels = [32,32,64,64, 128, 128, 128 ,128]
# n_classes = 10
# convnet = ConvNet(in_channels, channels, n_classes, padding=1)
# defconvnet = DeformConvNet(in_channels, channels, n_classes, padding=1)

# print(convnet(torch.rand((1,1,28,28))).shape)

# print(defconvnet(torch.rand(64,1,28,28)).shape)

In [5]:
class ConvNetArgus(Model):
    nn_module = ConvNet
    loss = 'CrossEntropyLoss'

class DeformConvNetArgus(Model):
    nn_module = DeformConvNet
    loss = 'CrossEntropyLoss'

In [6]:
PARAMS = {'nn_module': {
                        'in_channels': 1,
                         'channels': [32, 32, 64, 64, 128, 128, 128 ,128],
                         'n_classes': 10
                        },
          
           'optimizer': ('Adam', {'lr':1e-4}),
           'device':'cuda'
         }

In [7]:
model2 = DeformConvNetArgus(PARAMS)

In [8]:
model2.fit(mnist_trainset, val_loader=mnist_valset, max_epochs=100)

2019-08-18 19:13:21,409 INFO Validation, val_loss: 2.302594
2019-08-18 19:13:59,190 INFO Train - Epoch: 1, LR: 0.0001, train_loss: 1.734396
2019-08-18 19:14:02,331 INFO Validation - Epoch: 1, val_loss: 1.511404
2019-08-18 19:14:41,481 INFO Train - Epoch: 2, LR: 0.0001, train_loss: 1.495911
2019-08-18 19:14:44,809 INFO Validation - Epoch: 2, val_loss: 1.480819
2019-08-18 19:15:26,313 INFO Train - Epoch: 3, LR: 0.0001, train_loss: 1.479378
2019-08-18 19:15:29,592 INFO Validation - Epoch: 3, val_loss: 1.473534
2019-08-18 19:16:11,589 INFO Train - Epoch: 4, LR: 0.0001, train_loss: 1.474255
2019-08-18 19:16:14,860 INFO Validation - Epoch: 4, val_loss: 1.469982
2019-08-18 19:16:55,766 INFO Train - Epoch: 5, LR: 0.0001, train_loss: 1.472167
2019-08-18 19:16:59,117 INFO Validation - Epoch: 5, val_loss: 1.471421
2019-08-18 19:17:40,452 INFO Train - Epoch: 6, LR: 0.0001, train_loss: 1.471722
2019-08-18 19:17:43,825 INFO Validation - Epoch: 6, val_loss: 1.469036
2019-08-18 19:18:24,748 INFO Train

2019-08-18 19:53:09,591 INFO Validation - Epoch: 54, val_loss: 1.466342
2019-08-18 19:53:50,358 INFO Train - Epoch: 55, LR: 0.0001, train_loss: 1.464073
2019-08-18 19:53:53,727 INFO Validation - Epoch: 55, val_loss: 1.466036
2019-08-18 19:54:34,493 INFO Train - Epoch: 56, LR: 0.0001, train_loss: 1.463936
2019-08-18 19:54:37,798 INFO Validation - Epoch: 56, val_loss: 1.466306
2019-08-18 19:55:18,559 INFO Train - Epoch: 57, LR: 0.0001, train_loss: 1.464135
2019-08-18 19:55:21,959 INFO Validation - Epoch: 57, val_loss: 1.46655
2019-08-18 19:56:02,624 INFO Train - Epoch: 58, LR: 0.0001, train_loss: 1.464001
2019-08-18 19:56:06,038 INFO Validation - Epoch: 58, val_loss: 1.467072
2019-08-18 19:56:46,849 INFO Train - Epoch: 59, LR: 0.0001, train_loss: 1.46384
2019-08-18 19:56:50,217 INFO Validation - Epoch: 59, val_loss: 1.466898
2019-08-18 19:57:31,057 INFO Train - Epoch: 60, LR: 0.0001, train_loss: 1.463647
2019-08-18 19:57:34,350 INFO Validation - Epoch: 60, val_loss: 1.467014
2019-08-18 1

In [15]:
from dpipe.torch.utils import to_var, to_np
targets = []
prediction = []

for batch in mnist_valset:
    input, target = batch
    preds = model2.predict(input.cuda())
    prediction.extend(to_np(preds))
    targets.extend(to_np(target))

In [16]:
prediction = np.array(prediction)
targets = np.array(targets)

In [17]:
accuracy_score(targets, prediction.argmax(-1))

0.9898

In [18]:
model1 = ConvNetArgus(PARAMS)

In [19]:
model1.fit(mnist_trainset, val_loader=mnist_valset, max_epochs=20)

2019-08-18 18:38:49,226 INFO Validation, val_loss: 2.302661
2019-08-18 18:39:10,691 INFO Train - Epoch: 1, LR: 0.0001, train_loss: 1.882903
2019-08-18 18:39:12,762 INFO Validation - Epoch: 1, val_loss: 1.595926
2019-08-18 18:39:34,299 INFO Train - Epoch: 2, LR: 0.0001, train_loss: 1.530607
2019-08-18 18:39:36,365 INFO Validation - Epoch: 2, val_loss: 1.491429
2019-08-18 18:39:57,959 INFO Train - Epoch: 3, LR: 0.0001, train_loss: 1.489722
2019-08-18 18:40:00,038 INFO Validation - Epoch: 3, val_loss: 1.48318
2019-08-18 18:40:23,698 INFO Train - Epoch: 4, LR: 0.0001, train_loss: 1.479962
2019-08-18 18:40:25,865 INFO Validation - Epoch: 4, val_loss: 1.47548
2019-08-18 18:40:49,991 INFO Train - Epoch: 5, LR: 0.0001, train_loss: 1.476031
2019-08-18 18:40:52,186 INFO Validation - Epoch: 5, val_loss: 1.474442
2019-08-18 18:41:16,064 INFO Train - Epoch: 6, LR: 0.0001, train_loss: 1.473211
2019-08-18 18:41:18,246 INFO Validation - Epoch: 6, val_loss: 1.472171
2019-08-18 18:41:42,211 INFO Train -

In [20]:
from dpipe.torch.utils import to_var, to_np
targets = []
prediction = []

for batch in mnist_valset:
    input, target = batch
    preds = model1.predict(input.cuda())
    prediction.extend(to_np(preds))
    targets.extend(to_np(target))

In [21]:
prediction = np.array(prediction)
targets = np.array(targets)

In [22]:
accuracy_score(targets, prediction.argmax(-1))

0.9949